## 1.Import 및 라이브러리 다운로드

In [1]:
import gc
import torch
import sys

gc.collect()
torch.cuda.empty_cache()

In [2]:
!pip install rouge_score
!pip install datasets==1.0.2
!pip install transformers==4.24.0
!pip install transformer-utils
!pip install packaging
!pip install wandb

In [3]:
# 필요한 라이브러리 불러오기
import datasets
import transformers
import pandas as pd
from datasets import Dataset



import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import nltk

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments

)

from transformers import RobertaTokenizerFast
from transformers import EncoderDecoderModel
from dataclasses import dataclass, field
from typing import Optional
from datasets import load_dataset, load_from_disk
from tqdm import tqdm
from tabulate import tabulate

In [4]:
# 데이터 불러오기
# 20per data
train_20per = pd.read_csv('data/train_20per.csv')
val_20per = pd.read_csv('data/val_20per.csv')

# # 2~3sent
# train_2-3sent = pd.read_csv('data/train_2-3sent.csv')
# val_2-3sent = pd.read_csv('data/val_2-3sent.csv')

# # sum1
# train_Sum1 = pd.read_csv('data/train_Sum1.csv')
# val_Sum1 = pd.read_csv('data/val_Sum1.csv')


In [5]:
print(train_20per.count())
print(val_20per.count())

Text        73431
Summary     73431
Category    73431
dtype: int64
Text        9150
Summary     9150
Category    9150
dtype: int64


In [6]:
train_20per['Category'].value_counts()

speech        16000
minute        13600
news_r        10800
briefing       8000
literature     4800
narration      4231
his_cul        4000
paper          4000
edit           4000
public         4000
Name: Category, dtype: int64

In [7]:
train_20per.head()

,Text,Summary,Category
0,"지어내버린 대목부터는 흥분이 버썩 줄어지었다 ──. ""선생님! 또 기침이 나고 토...","그래서 나는 곧 H에게 간단하고 힘 있는 답장을 썼다 ──. ""선생님! 또 기침이 ...",literature
1,이 송아지가 젖을 떼우고 집으로 끌고 오던 날은 첨지는 개선장군이 성안 에 들어올...,이 송아지가 젖을 떼우고 집으로 끌고 오던 날은 첨지는 개선장군이 성안 에 들어올 ...,literature
2,어떤 여름날 밤 손주딸에게 관한 불길한 꿈을 꾼 이 한머니는 이튿날 조반 후에 생...,어떤 여름날 밤 손주딸에게 관한 불길한 꿈을 꾼 이 한머니는 이튿날 조반 후에 생각...,literature
3,이러한 가운데서 왕후는 자기의 입장을 위태롭게 여기고 겸하여 장래 자기 의 몸으로...,자기는 태자의 위 따위는 부럽지 않다. 이 전 낙랑공주가 아직 살아 있고 그 낙랑공...,literature
4,“가겠소.” “언니가 나오시면 일러드 리겠으니 그때까지는 찾아오지 않으시는 것이 ...,별안간 골목쟁이에서 쑥 내달아 붙잡지나 않을까를 염려하여 빠른 걸음으 로 골목 골목...,literature


In [8]:
train_20per = train_20per.loc[(train_20per.Category != 'literature')& 
               (train_20per.Category != 'minute')&
               (train_20per.Category != 'edit')&
               (train_20per.Category != 'speech')]

In [9]:
val_20per = val_20per.loc[(val_20per.Category != 'literature')& 
               (val_20per.Category != 'minute')&
               (val_20per.Category != 'edit')&
               (val_20per.Category != 'speech')]

In [10]:
train_20per.head()

,Text,Summary,Category
4800,방문객이 상상으로 연주하는 곳.\n쇼핑몰과 기념품 가게가 없는 우주피스 공화국의 거...,쇼핑몰과 기념품 가게가 없는 우주피스 공화국의 거리에는 공연과 전시를 홍보하는 포스...,narration
4801,저희 입장에서도 사실 이번 선거를 저희는 문재인 정부가 이제 한 2년 남짓 남았습니...,저희 입장에서도 사실 이번 선거를 저희는 문재인 정부가 이제 한 2년 남짓 남았습니...,narration
4802,이제 뭐 훈련된 익숙한 인력이 투입이 돼야 되는데 이게 쉽지가 않은 거죠. 그래서 ...,게다가 지금의 발생양상이 지속되거나 더 나빠진다면 한 1~2주 정도 지나면 중환자 ...,narration
4803,하지만 지금은 연어가 아닌 산천어가 이 곳의 주인이 되었다.\n그 옛날 바다로 나가...,금강 최상류에 위치한 용담댐. 2001년 건설된 댐으로 우리나라에서 다섯번째로 큰 ...,narration
4804,티 없이 맑은 어린아이.\n금발의 젊은 여성.\n그리고 말을 타고 해변을 산책하는 ...,피우라 시에서 직접 운영하는 박물관을 찾았다. 비쿠스 왕조의 금 장식물들이 유명한 ...,narration


In [11]:
train_20per= train_20per.drop(columns='Category')
val_20per = val_20per.drop(columns='Category')

In [12]:
train_20per.reset_index(inplace=True, drop=True)
val_20per.reset_index(inplace=True, drop=True)

In [13]:
train_20per.head()

,Text,Summary
0,방문객이 상상으로 연주하는 곳.\n쇼핑몰과 기념품 가게가 없는 우주피스 공화국의 거...,쇼핑몰과 기념품 가게가 없는 우주피스 공화국의 거리에는 공연과 전시를 홍보하는 포스...
1,저희 입장에서도 사실 이번 선거를 저희는 문재인 정부가 이제 한 2년 남짓 남았습니...,저희 입장에서도 사실 이번 선거를 저희는 문재인 정부가 이제 한 2년 남짓 남았습니...
2,이제 뭐 훈련된 익숙한 인력이 투입이 돼야 되는데 이게 쉽지가 않은 거죠. 그래서 ...,게다가 지금의 발생양상이 지속되거나 더 나빠진다면 한 1~2주 정도 지나면 중환자 ...
3,하지만 지금은 연어가 아닌 산천어가 이 곳의 주인이 되었다.\n그 옛날 바다로 나가...,금강 최상류에 위치한 용담댐. 2001년 건설된 댐으로 우리나라에서 다섯번째로 큰 ...
4,티 없이 맑은 어린아이.\n금발의 젊은 여성.\n그리고 말을 타고 해변을 산책하는 ...,피우라 시에서 직접 운영하는 박물관을 찾았다. 비쿠스 왕조의 금 장식물들이 유명한 ...


In [14]:
print(train_20per.count())
print(val_20per.count())

Text       35031
Summary    35031
dtype: int64
Text       4350
Summary    4350
dtype: int64


In [15]:
# 괄호까지 없앨수 있도록 구현해야 겠음, 결과물에서 괄호가 너무 방해됨
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (...) 제거
    sentence = re.sub("[()]","", sentence)
    sentence = re.sub('"','', sentence) # 쌍따옴표 제거
    sentence = re.sub("'",'', sentence) # 따옴표 제거
    sentence = re.sub('\n','', sentence) # \n " 제거
    sentence = re.sub('.{2,3}\W{0,1}기자','', sentence) # 기자 이름 제거
    sentence = re.sub(r'[?.!,][/?.!,]', '', sentence) # 여러개 문장 부호를 하나의 문장부호로 바꿉니다
    sentence = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣a-z0-9]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거

    return sentence


In [16]:
clean_text = []
clean_headlines = []

for i in tqdm(train_20per['Text']):
    clean_text.append(preprocess_sentence(i))
    
for i in tqdm(val_20per['Summary']):
    clean_headlines.append(preprocess_sentence(i))

100%|██████████| 4350/4350 [00:00<00:00, 20409.03it/s]


In [17]:
print(clean_text[:10])

['방문객이 상상으로 연주하는 곳 쇼핑몰과 기념품 가게가 없는 우주피스 공화국의 거리에는 공연과 전시를 홍보하는 포스터들이 옹기종기 붙어 있다 이곳은 단순한 예술가 거리가 아니다 독자적인 헌법이 있는 엄연한 자칭 공화국이다 41개의 헌법 조항에 차마 포함되지 못한 예민한 사항들은 이렇게 거리에서 논의되기도 한다 여의도 면적의 5분의 1밖에 안되는 작은 공화국에는 우체국을 겸하는 입국심사대도 있다 이곳에서 무료로 도착 비자가 발급된다 이 사람은 우주피스의 대통령 로마스 릴레이키스다 영화감독인 그는 1997년 이 지역에 살던 예술가들과 함께 아무도 인정하지 않지만 누구나 오고 싶은 나라를 만들었다 빌뉴스에서 북서쪽으로 002시간 반을 달려 도착한 샤 울랴이시 근교의 시골 마을 집도 한 채 안 보이는 허허발판에', '저희 입장에서도 사실 이번 선거를 저희는 문재인 정부가 이제 한 2년 남짓 남았습니다 그동안 소위 대통령 탄핵이라고 하는 그런 아주 비정상적 상황 속에서 출발한 정권이거든요 나름의 그 어려움을 뚫고 지금까지 와 있는데 물론 부족하고 모자란 측면도 많이 있을 겁니다 또 나머지 임기 2년을 무사히 또 현재 이런 코로나 경제위기와 같은 이런 어려운 상황을 뚫고 나가려면 국민들이 이번 선거에서 보다 더 연료를 채워주시는 그래서 저희는 중간 급유를 해주는 그런 선거다 이렇게 좀 생각을 하는데 그런 면에서 투표률이 저희도 좀 높아지길 원하고 있습니다 근데 엊그제 다행스러운 건 여론조사 결과를 보니까 지난 총선 대비 투표를 하겠다고 하는 의향률이 한 10 이상 올라갔습니다 우리 걱정과는 달리 그래서 저희는 이번 과정을 보면서 뭔가 이런 경제적인 위기나 어려움 속에서 국가의 존재가 무엇이어야 되는가 뭐 이런 거와 관련해서 국민들의 관심이 이제 높아진 것 같습니다 재난기본지원금 재난 그 지원금 이런 것들 때문에', '이제 뭐 훈련된 익숙한 인력이 투입이 돼야 되는데 이게 쉽지가 않은 거죠 그래서 단기간에 확충이 좀 어려운 면이 있는데 사실은 지금 발생한 환자 숫자만 하

In [18]:
train_20per['Text'] = clean_text
val_20per['Summary'] = clean_headlines

In [19]:
# DF > data Set으로 전환
train_data = Dataset.from_pandas(train_20per) 
val_len = len(val_20per) // 2
val_data = Dataset.from_pandas(val_20per[:val_len])
test_data=Dataset.from_pandas(val_20per[val_len:])

In [20]:
# train_len = len(train_20sent)//4
# train_data = Dataset.from_pandas(train_20sent[:train_len]) 
# val_len = len(val_20sent) // 8
# val_data = Dataset.from_pandas(val_20sent[:val_len])
# test_data=Dataset.from_pandas(val_20sent[val_len:])

In [21]:
print(train_data)
print(val_data)
print(test_data)

Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 35031)
Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 2175)
Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 2175)


In [22]:
max_input = 512
max_target = 128
batch_size = 4
model_checkpoints = "/aiffel/aiffel/Aiffelthon_koBART/results221111/checkpoint-21500"

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)

In [24]:
def preprocess_data(data_to_process):
  #get all the dialogues
  inputs = [dialogue for dialogue in data_to_process['Text']]
  #tokenize the dialogues
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)
  #tokenize the summaries
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['Summary'], max_length=max_target, padding='max_length', truncation=True)
    
  #set labels
  model_inputs['labels'] = targets['input_ids']
  #return the tokenized data
  #input_ids, attention_mask and labels
  return model_inputs

In [25]:
train_tokenize_data = train_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])
val_tokenize_data = val_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])

/opt/conda/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/36 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [26]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [27]:
# # set special tokens
# #from transformers import EncoderDecoderConfig
# model.config.decoder_start_token_id = tokenizer.bos_token_id                                             
# model.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
model.config.max_length = 128
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 2
model.config.length_penalty = 2.0
model.config.num_beams = 2


In [28]:
model.config

BartConfig {
  "_name_or_path": "/aiffel/aiffel/Aiffelthon_koBART/results221111/checkpoint-21500",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 2,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "NEGA

In [29]:
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),}

In [30]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results221112",
    num_train_epochs=5,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,  # demo
    per_device_eval_batch_size=16,
    learning_rate=3e-05,
    warmup_steps=2000,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True, # 생성기능을 사용하고 싶다고 지정한다.
    logging_dir="logs2",
    logging_steps=2000,
    save_total_limit=3,
)

In [31]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) # 데이터 일괄 처리?
"""
DataCollatorForSeq2Seq 를 사용하여 예제 배치를 생성 하십시오 . 
또한 일괄 처리에서 가장 긴 요소의 길이로 텍스트와 레이블을 동적으로 채워서 균일한 길이가 되도록 합니다.
tokenizer를 설정하여 함수 에서 텍스트를 채울 수 있지만 padding=True동적 패딩이 더 효율적입니다.
"""

'\nDataCollatorForSeq2Seq 를 사용하여 예제 배치를 생성 하십시오 . \n또한 일괄 처리에서 가장 긴 요소의 길이로 텍스트와 레이블을 동적으로 채워서 균일한 길이가 되도록 합니다.\ntokenizer를 설정하여 함수 에서 텍스트를 채울 수 있지만 padding=True동적 패딩이 더 효율적입니다.\n'

In [32]:
trainer = Seq2SeqTrainer(
    model, 
    training_args,
    train_dataset=train_tokenize_data,
    eval_dataset=val_tokenize_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [33]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 35031
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 10950
  Number of trainable parameters = 123859968
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: jx7789. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
2000,1.449000
4000,1.445800
6000,1.433900
8000,1.423600
10000,1.416300


Saving model checkpoint to results221112/checkpoint-500
Configuration saved in results221112/checkpoint-500/config.json
Model weights saved in results221112/checkpoint-500/pytorch_model.bin
tokenizer config file saved in results221112/checkpoint-500/tokenizer_config.json
Special tokens file saved in results221112/checkpoint-500/special_tokens_map.json
Saving model checkpoint to results221112/checkpoint-1000
Configuration saved in results221112/checkpoint-1000/config.json
Model weights saved in results221112/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in results221112/checkpoint-1000/tokenizer_config.json
Special tokens file saved in results221112/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to results221112/checkpoint-1500
Configuration saved in results221112/checkpoint-1500/config.json
Model weights saved in results221112/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in results221112/checkpoint-1500/tokenizer_config.json
Special tok

TrainOutput(global_step=10950, training_loss=1.4319670488192067, metrics={'train_runtime': 13864.5228, 'train_samples_per_second': 12.633, 'train_steps_per_second': 0.79, 'total_flos': 5.33991946715136e+16, 'train_loss': 1.4319670488192067, 'epoch': 5.0})

In [35]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2175
  Batch size = 16


{'eval_loss': 2.1757662296295166,
 'eval_rouge2_precision': 0.1236,
 'eval_rouge2_recall': 0.2408,
 'eval_rouge2_fmeasure': 0.149,
 'eval_runtime': 408.8078,
 'eval_samples_per_second': 5.32,
 'eval_steps_per_second': 0.333,
 'epoch': 5.0}

In [ ]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["Text"],
        padding="max_length",
        truncation=True,
        max_length=max_target,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)# 여기에 기본 kobart가져오기?

test_samples = val_data.select(range(16))

summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1] # 여기에 체크포인트 가져오기 
# 연구해봐야한다.

In [ ]:
# from tabulate import tabulate

In [ ]:
for i in range(len(summaries_after_tuning)):
    print('idx_{} '.format(i))
    print("Summary before \n", summaries_before_tuning[i])
    print()
    print("Summary after \n", summaries_after_tuning[i])
    print()
    print("Target summary \n", test_samples["Summary"][i])
    print()
    print('Text', test_samples["Text"][i])
    print('-'*100)
    print()
    

In [ ]:
# print(
#     tabulate(
#         zip(
#             range(len(summaries_after_tuning)),
#             summaries_after_tuning,
#             summaries_before_tuning,
#         ),
#         headers=["Id", "Summary after", "Summary before"]
#     )
# )
# print("\nTarget summaries:\n")
# print(
#     tabulate(list(enumerate(test_samples["Summary"])), headers=["Id", "Target summary"])
# )
# print("\nSource documents:\n")
# print(tabulate(list(enumerate(test_samples["Text"])), headers=["Id", "Text"]))

In [ ]:
# 첫번재 시도 : max_len128로 학습, 10 epoch, lr 3e-05 ,target_len 32, results_ml128 폴더 생성
# 두번째 시도 : max_len128로 학습, 10 epoch, lr 4e-05 ,target_len 32, results_ml128_2 폴더 생성
# 세번째 시도 : max_len512로 학습, 2 epoch, lr 5e-05 ,target_len 128
# 네번째 시도 : max_len512로 학습, 3 epoch, lr 5e-05 ,target_len 128
# 모델 바꿈 : noahkim/KoT5_news_summarization, https://huggingface.co/digit82/kobart-summarization